In [36]:
from env import get_db_url
import pandas as pd
import numpy as np
import os

In [37]:
def get_store_data(use_cache = True):
    """
    Get store data from the database.
    """
    filename = "store_data.csv"
    if os.path.isfile(filename) and use_cache:
        print("Loading store data from cache...")
        store_data = pd.read_csv(filename)
    print("reading store data from database...")
    url = get_db_url('superstore_db')
    query = '''
    SELECT  orders.*,
        categories.`Category`, 
        categories.`Sub-Category`,
        customers.`Customer Name`,
        products.`Product Name`,
        regions.`Region Name`
    FROM orders
    LEFT JOIN categories USING (`Category ID`)
    LEFT JOIN customers USING (`Customer ID`)
    LEFT JOIN products USING (`Product ID`)
    LEFT JOIN regions USING (`Region ID`)
    '''
    df = pd.read_sql(query, url)
    print("writing store data to cache...")
    df.to_csv(filename, index = False)
    return df

In [38]:
df = get_store_data(use_cache=False)
df.head()

reading store data from database...
writing store data to cache...


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,...,Quantity,Discount,Profit,Category ID,Region ID,Category,Sub-Category,Customer Name,Product Name,Region Name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,2.0,0.00,41.9136,1,1,Furniture,Bookcases,Claire Gute,Bush Somerset Collection Bookcase,South
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,3.0,0.00,219.5820,2,1,Furniture,Chairs,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",South
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,...,2.0,0.00,6.8714,3,2,Office Supplies,Labels,Darrin Van Huff,Self-Adhesive Address Labels for Typewriters b...,West
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,5.0,0.45,-383.0310,4,1,Furniture,Tables,Sean O'Donnell,Bretford CR4500 Series Slim Rectangular Table,South
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,2.0,0.20,2.5164,5,1,Office Supplies,Storage,Sean O'Donnell,Eldon Fold 'N Roll Cart System,South


In [39]:
# rename all columns to lowercase
df.columns = [c.lower() for c in df.columns]
# replace spaces in column names with underscores
df.columns = [c.replace(' ', '_') for c in df.columns]
# replace - with _ in column names
df.columns = [c.replace('-', '_') for c in df.columns]
# convert the date columns to datetime
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])


In [41]:
df.isnull().sum()
#no null values

order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
segment          0
country          0
city             0
state            0
postal_code      0
product_id       0
sales            0
quantity         0
discount         0
profit           0
category_id      0
region_id        0
category         0
sub_category     0
customer_name    0
product_name     0
region_name      0
dtype: int64

In [60]:
df.sales.min(), df.sales.max()

(1.0799999999999998, 8159.951999999998)

In [65]:
# create a new df for items, quantity, and price, discount
df_items = df[['order_id', 'product_id', 'quantity', 'discount', 'sales', 'profit']].copy()
#add unit cost column
df_items['unit_cost'] = df_items.sales / ((1-df_items.discount) * df_items.quantity)
#add unit profit column
df_items['unit_profit'] = df_items.profit / df_items.quantity
df_items.head()




,order_id,product_id,quantity,discount,sales,profit,unit_cost,unit_profit
0,CA-2016-152156,FUR-BO-10001798,2.0,0.00,261.9600,41.9136,130.98,20.9568
1,CA-2016-152156,FUR-CH-10000454,3.0,0.00,731.9400,219.5820,243.98,73.1940
2,CA-2016-138688,OFF-LA-10000240,2.0,0.00,14.6200,6.8714,7.31,3.4357
3,US-2015-108966,FUR-TA-10000577,5.0,0.45,957.5775,-383.0310,348.21,-76.6062
4,US-2015-108966,OFF-ST-10000760,2.0,0.20,22.3680,2.5164,13.98,1.2582


In [66]:
df_items.describe()

,quantity,discount,sales,profit,unit_cost,unit_profit
count,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000
mean,3.794694,0.160110,229.206358,21.880351,74.890006,6.197098
std,2.297407,0.206419,532.862614,202.891949,175.232540,49.454376
min,1.000000,0.000000,1.080000,-3839.990400,1.240000,-959.997600
25%,2.000000,0.000000,16.695000,1.702400,6.480000,0.722800
50%,3.000000,0.200000,52.245000,8.073050,19.990000,2.660250
75%,5.000000,0.200000,209.917500,27.414450,79.872500,8.158600
max,14.000000,0.800000,8159.952000,3177.475000,3999.990000,997.995000


In [77]:
df[['product_id', 'product_name']].sort_values('product_name')

,product_id,product_name
1708,OFF-PA-10003424,"""While you Were Out"" Message Book, One Form pe..."
355,OFF-EN-10001137,"#10 Gummed Flap White Envelopes, 100/Box"
985,OFF-EN-10000461,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes"
1228,OFF-EN-10001219,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes"
711,OFF-EN-10001219,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes"
...,...,...
1679,TEC-PH-10003589,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...
1612,TEC-PH-10002726,netTALK DUO VoIP Telephone Service
1696,TEC-PH-10002726,netTALK DUO VoIP Telephone Service
1672,TEC-PH-10002726,netTALK DUO VoIP Telephone Service


In [71]:
df.discount.value_counts()

0.00    810
0.20    656
0.70     82
0.80     46
0.40     41
0.30     31
0.60     19
0.50     18
0.10     14
0.15      8
0.45      5
0.32      4
Name: discount, dtype: int64

In [78]:
# add a column for brand
df['brand'] = df.product_name.str.split(' ').str[0]
df.head()


,order_id,order_date,ship_date,ship_mode,customer_id,segment,country,city,state,postal_code,...,discount,profit,category_id,region_id,category,sub_category,customer_name,product_name,region_name,brand
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,0.00,41.9136,1,1,Furniture,Bookcases,Claire Gute,Bush Somerset Collection Bookcase,South,Bush
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,...,0.00,219.5820,2,1,Furniture,Chairs,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",South,Hon
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,...,0.00,6.8714,3,2,Office Supplies,Labels,Darrin Van Huff,Self-Adhesive Address Labels for Typewriters b...,West,Self-Adhesive
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,0.45,-383.0310,4,1,Furniture,Tables,Sean O'Donnell,Bretford CR4500 Series Slim Rectangular Table,South,Bretford
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,...,0.20,2.5164,5,1,Office Supplies,Storage,Sean O'Donnell,Eldon Fold 'N Roll Cart System,South,Eldon


In [80]:
df.brand.value_counts()

Xerox              133
Avery               95
GBC                 54
Newell              45
Eldon               43
                  ... 
Companion            1
Fluorescent          1
Magnifier            1
Logitech Gaming      1
Avanti               1
Name: brand, Length: 371, dtype: int64

In [ ]:
# # round sales and profit to 2 decimal places
# df['sales'] = df.sales.round(2)
# df['profit'] = df.profit.round(2)


In [82]:
# how many brands have more than 1 product
df.brand.value_counts()[df.brand.value_counts() > 10]

Xerox          133
Avery           95
GBC             54
Newell          45
Eldon           43
Global          43
Fellowes        42
Acco            34
Wilson          33
Logitech        31
Hon             27
Belkin          26
Howard          17
Boston          15
Cisco           15
Safco           14
Ibico           14
Acme            14
Tenex           13
Advantus        13
Bush            13
Staples         12
Bevis           12
Staple          12
Recycled        12
Plantronics     11
Kensington      11
DAX             11
Panasonic       11
Cardinal        11
Name: brand, dtype: int64